### 금융 문서 data에서 초해상화 학습을 위한 데이터 생성
  - input: 다양한 size의 서류 이미지지
  - output: hr 128, lr 64의 텍스트

### onedrive mounting

In [ ]:
!wget https://downloads.rclone.org/v1.57.0/rclone-v1.57.0-linux-amd64.deb
!apt install ./rclone-v1.57.0-linux-amd64.deb

In [ ]:
!rclone config

In [ ]:
!mkdir onedrive
!nohup rclone --vfs-cache-mode writes mount onedrive: ./onedrive &    

nohup: appending output to 'nohup.out'


### google drive mounting

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### train_downscaling

In [ ]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import math
import zipfile

In [ ]:
HR_SIZE = 128
LR_SIZE = 64

In [ ]:
train_zip = '/content/onedrive/playdata/final_project/financial/train/financial_ocr_ts1.zip'

with zipfile.ZipFile(train_zip, 'r') as zf:
	zf.extractall('/content')

In [ ]:
img_list = os.listdir('/content/result/bank/images')

In [ ]:
cnt = 0
now = 'train'

In [ ]:
dic = {'val': {'lr':[], 'hr':[]}, 'train':{'lr':[], 'hr':[]}}

In [ ]:
with zipfile.ZipFile(r'/content/drive/MyDrive/final_project/croptext.zip', 'w', compression=zipfile.ZIP_DEFLATED) as new_zip:
    for filename in img_list:
      if cnt >= 9999: break
      img = cv2.imread(f'/content/result/bank/images/{filename}', cv2.IMREAD_GRAYSCALE)
      y, x = img.shape
      for i in range(0, x-HR_SIZE, HR_SIZE):
        for j in range(0, y-HR_SIZE, HR_SIZE):
          if cnt >= 9999: break
          elif cnt >= 8000:
            now = 'val'
          cnt += 1
          path_img_lr = f'{now}_lr/{str(cnt).zfill(4)}.jpg'
          path_img_hr = f'{now}_hr/{str(cnt).zfill(4)}.jpg'
          cv2.imwrite(path_img_lr, cv2.resize(img[j:j+HR_SIZE, i:i+HR_SIZE], (LR_SIZE, LR_SIZE), interpolation=cv2.INTER_AREA))
          cv2.imwrite(path_img_hr, img[j:j+HR_SIZE, i:i+HR_SIZE])
          new_zip.write(path_img_lr)
          new_zip.write(path_img_hr)
          dic[now]['lr'].append(path_img_lr)
          dic[now]['hr'].append(path_img_hr)

In [ ]:
len(os.listdir('val_lr'))

1930

In [ ]:
len(os.listdir('val_hr'))

1930

In [ ]:
pd.DataFrame(dic['val']).to_csv('/content/drive/MyDrive/final_project/croptext_val.csv')
pd.DataFrame(dic['train']).to_csv('/content/drive/MyDrive/final_project/croptext_train.csv')